In [ ]:
import warnings

warnings.filterwarnings('ignore')

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer()
optimizer.get_all_study_names()


In [ ]:
study_name = "macd_bb_with_optimizer"
df = optimizer.get_study_trials_df(study_name)
df

In [ ]:
df_filtered = df[(df["value"] > 0.5) & (df["total_executors"] > 40)].sort_values("total_executors", ascending=False)
df_filtered

In [ ]:
import plotly.express as px

px.scatter(df_filtered, x="total_executors", y="sharpe_ratio", color="net_pnl", hover_data=["number"])